# Modulo 2: Saper leggere e rappresentare i dati

Quest'oggi partiamo direttamente con l'utilizzo di un dataset 🚀. 

Il dataset con cui lavoreremo è un dataset contenente informazioni sulle **emissioni** prodotte dai singoli paesi dal 1960 al 2018 (**i valori sono espressi in tonnellate di CO2**).
La fonte dei dati è la [banca dati](https://data.worldbank.org/) della Banca Mondiale.

Come la scorsa volta iniziamo con il caricare tutte le librerie che ci servono.

Carichiamo anche una funzione della libreria `google.colab`, `data_table` che ci semplifica la vita se vogliamo "navigare" i dati, ovvero ispezionarli.

 


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import data_table
data_table.enable_dataframe_formatter()


Per prima cosa carichiamo il database che utilizzeremo oggi e diamogli una sbirciata.

In [ ]:
emissions = pd.read_csv('https://raw.githubusercontent.com/federicadaniele/PCTOcodingdatascience/main/Modulo2/emissions_WB.csv')
emissions.head(100)

In [ ]:
# il massimo che sono riuscita a trovare di distribuzione simmetrica in questo DB 
emissions_ITA = emissions[emissions['countryname'] == 'Italy'] 
emissions_ITA.hist('population',bins=10)

In [ ]:
emissions_ITA.head(10)

Iniziamo la cosiddetta esplorazione dei dati.

Per cominiciare, cos'è che secondo voi identifica un'osservazione in questo database? 

Quali informazioni (o **variabili**) abbiamo su ciascuna osservazione? Che **tipologia di dati abbiamo**?

Quante **osservazioni**?

**A voi il 🎤!**




In [ ]:
# COMPLETA!

Quanti sono i paesi in tutto inclusi nel dataset?

In [ ]:
len(pd.unique(emissions['countrycode']))

Il database presenta dei dati mancanti?

In [ ]:
emissions.count()

Può essere utile vedere in quali anni si concentrano i dati mancanti:

In [ ]:
emissions['emissions'].groupby(emissions['year']).count()

Ok, quindi sostanzialmente nel 1990 si aggiungono 20 paesi al dataset. Al fine di mantenere il gruppo di paesi sotto osservazione costanti nel tempo, rimuoviamo i paesi per i quali non abbiamo i dati completi.

In [ ]:
conteggio = emissions['emissions'].groupby(emissions['countrycode']).count().reset_index(name='count')
emissions = pd.merge(emissions,conteggio,on=['countrycode'],validate="many_to_one",how="inner",indicator=True)
emissions = emissions[emissions['count']==59]
emissions.count()

Quanti sono adesso i paesi inclusi nel database?

In [ ]:
# COMPLETA!

Due diligence terminata ✅. Entriamo nel vivo della lezione di oggi. 
 

# Parte A: Saper leggere i dati. Basi di statistica.

Abbiamo visto che abbiamo quasi 12.000 osservazioni, ovvero **un'osservazione per ogni paese e anno dal 1960 al 2018**: di ciascun paese in ciascun anno conosciamo le emissioni di CO2 complessive (espresse in tonnellate), la popolazione, e le emissioni **pro capite**.

Qualcuno che conosce il significato dell'espressione "pro capite"?

Cominciamo con l'analisi empirica vera e propria e con alcune **statistiche descrittive** che ci aiutano a leggere meglio questi dati:

1.   Media
2.   Minimo e massimo
3.   Varianza
4.   P90/P10

Partiamo dalla media: 

$\overline{x}=\frac{1}{N} \sum_i x_i$


In [ ]:
emissions['emissions'].mean()

Quindi a quanto ammonta il valore medio delle emissioni prodotte dal 1960 ad oggi dai singoli paesi? A voi il 🎤!

Quali sono le altre statistiche descrittive che abbiamo visto? Calcolarle in `python` è molto più semplice che farlo con la calcolatrice. Pensa alla varianza:
$ \sigma^2 = \frac{1}{N} \sum_i (x_i - \overline{x})^2 $ 

Il comando che ci consente di calcolarle in `python` è molto semplice:

```
# database['variabile'].stat()
```

dove `.stat()` può essere `.mean()`, `.var()`, `.min()`, `.max()`, `.quantile(q=0.10)`, `.quantile(q=0.90)`.



**Domanda**: i diversi paesi hanno prodotto **in media** più o meno emissioni nel 2018 rispetto al 1960? 

Per rispondere a questa domanda ci serve calcolare la media delle emissioni in ciascun anno, che possiamo fare con `groupby`, una funzione molto utile che ci consente di calcolare media, varianza, etc. di una determinata variabile (chiamiamola "variabile1") per **sottogruppi**, dove chiamiamo "variabile2" la variabile che identifica i sottogruppi:

La struttura del comando per calcolare **statistiche descrittive per sottogruppi** è:
```
# database['variabile1'].groupby(database['variabile2']).stat()
```
 
Applichiamo la regola per trovare il valore medio delle emissioni in ciascun anno:



In [ ]:
emissions['emissions'].groupby(emissions['year']).mean()

In [ ]:
# per conoscere invece il totale?
emissions['emissions'].groupby(emissions['year']).sum()/1000000000

**Esercizio** 💡: riuscite a scrivere un semplice comando per calcolare più o meno il tasso di crescita delle emissioni medie dal 1960 al 2018?

**Suggerimento**: vi conviene fare uno step intermedio e creare un dataset intermedio in cui salvate le medie annuali.

In [ ]:
# COMPLETA!

Un modo alternativo e più rigoroso è quello di sfruttare l'**indicizzazione** delle osservazioni all'interno di un dataframe. In sostanza, ciascun dataframe dispone di un **indice** che identifica univocamente un'osservazione all'interno del database.

In [ ]:
medie = emissions['emissions'].groupby(emissions['year']).mean()
medie.head()

Per esempio: vogliamo conoscere l'ammontare delle emissioni medie nel 2018? Possiamo farlo chiedendo di "stampare" la riga del database associata all'indice = 2018:

In [ ]:
print(medie[2018])

Riuscite a trovare un modo alternativo per calcolare il tasso di crescita delle emissioni medie annue dal 1960 al 2018 sfruttando l'indicizzazione dei dati?

In [ ]:
# COMPLETA!

**Domanda:** cambia qualcosa se al posto delle emissioni totali di ciascun paese utilizziamo le **emissioni pro capite?**

In [ ]:
medie = emissions['emissions_per_capita'].groupby(emissions['year']).mean()
# COMPLETA!

**Secondo voi ** se uno volesse sapere quale paese inquina di più, quale delle due variabili gli converrebbe utilizzare, emissioni o emissioni pro capite?

Andiamo avanti... 
Durante l'analisi di un database, **potrebbe essere utile sapere come selezionare soltanto alcune righe o osservazioni** per fare delle analisi solo su queste. 
Un modo per selezionare alcune righe specifiche lo abbiamo già visto, è quello dell'indicizzazione, ma esiste un metodo più efficiente, che ci consente di **selezionare tutti i dati che rispettano una determinata condizione**.

La struttura per farlo è la seguente:

```
# database[database['variabile']=='condizione']
```

Vediamo un esempio. Vogliamo selezionare i dati sulle emissioni riferiti soltanto al 2018:

In [ ]:
emissions2018 = emissions[emissions['year'] == 2018]

Per esempio vogliamo conoscere **qual'è il paese che nel 2018 aveva un valore di emissioni pro capite uguale al valore minimo/massimo:**

In [ ]:
# calcoliamo prima il valore minimo e massimo:
minempc = emissions2018['emissions_per_capita'].min()
maxempc = emissions2018['emissions_per_capita'].max()

# e poi "sezioniamo" i dati chiedendo di selezionare soltanto le osservazioni che rispettano la condizione: quale condizione?
emissions2018[emissions2018['emissions_per_capita'] == maxempc] 

In [ ]:
# COMPLETA!

Durante la presentazione abbiamo parlato del concetto di **variabilità dei dati**. Non tutti i paesi inquinano allo stesso modo: abbiamo visto come banalmente alcuni paesi sono più popolosi e inquinano di più.

Ad esempio, **nel 2018 il 10% di paesi con i valori più alti di emissioni pro capite inquinavano quasi 38 volte tanto quanto inquinavano il 10% di paesi con i valori più bassi di emissioni pro capite:**

In [ ]:
emissions2018['emissions_per_capita'].quantile(q=0.90)/emissions2018['emissions_per_capita'].quantile(q=0.10)

Un altro modo abbastanza intuitivo per riassumere la variabilità dei dati è quella di calcolare il **coefficiente di variazione**:

$CV = \sigma / \overline{x}$

ovvero il rapporto tra la deviazione standard (la radice quadrata della varianza) e la media.

Il coefficiente di variazione ci dice quanto sono diversi tra loro i dati relativi ad una determinata variabile in relazione al valore medio di quella determinata variabile (quest'ultima operazione di dividere per il valore medio di una variabile si chiama anche **normalizzazione**). **Tanto maggiore è il coefficiente di variazione quindi, tanto maggiore è la variabilità dei dati.**

Per esempio, potremmo domandarci qual'è il coefficiente di variazione complessivo dei nostri dati nel 2018?

**Esercizio** 💡: vuoi provare a rispondere all'ultima domanda?

In [ ]:
medie = emissions2018['emissions_per_capita'].mean()
sigma2 = emissions2018['emissions_per_capita'].var()

# utilizziamo numpy:
sigma = np.sqrt(sigma2)
CV = medie/sigma
print(CV)

La variabilità delle emissioni pro capite la possiamo vedere anche con l'aiuto di un **istogramma**, che ci consente di visualizzare la **distribuzione** di una determinata variabile.

In [ ]:
emissions2018.hist('emissions_per_capita',bins=100)

# Parte B: Saper rappresentare i dati
Prima abbiamo visto di quanto è aumentata in media la produzione di emissioni nel corso degli anni. Questa informazione la possiamo inserire in una tabella se stiamo facendo una presentazione. Tuttavia un **grafico** è spesso un modo più efficace di comunicare.

In [ ]:
emissions['media'] = emissions['emissions'].groupby(emissions['year']).mean() 
emissions['media'].plot()

Questo particolare grafico ci consente di visualizzare quella che in gergo si chiama una **serie storica**, ovvero una determinata variabile osservata nel corso del tempo.

Ora costruisci un grafico analogo ma applicato alle emissioni pro capite. **Trovi delle differenze?**

In [ ]:
# COMPLETA!

Un altro modo utile di visualizzare i dati è quello del **grafico a barre.** Per esempio vogliamo vedere a quanto ammontavano le emissioni pro capite nel 2018 in Italia, Francia e Cina a confronto.

In [ ]:
emissionssubset = emissions[(emissions['countrycode']=='ITA')|(emissions['countrycode']=='FRA')|(emissions['countrycode']=='CHN')]
emissionssubset = emissionssubset[(emissionssubset['year']==1960)|(emissionssubset['year']==1980)|(emissionssubset['year']==2000)|(emissionssubset['year']==2018)]

emissionssubset = emissionssubset[['year','countrycode','emissions_per_capita']]
pivot =  emissionssubset.pivot(index="year", columns="countrycode", values="emissions_per_capita")

pivot.head()
ax = pivot.plot.bar(rot=0)

**Cosa notate?** 🎤 a voi!

Un ultimo tipo di grafico che vediamo è un grafico molto utilizzato, il cosiddetto **scatter plot**. Lo scatter plot serve principalmente per visualizzare la correlazione tra due variabili numeriche. Una variabile verrà visualizzata sull'asse delle y, mentre l'altra sull'asse delle x. 

Per esempio, possiamo visualizzare sull'asse delle y il valore delle emissioni pro capite e su quello delle x la popolazione complessiva. Lo possiamo fare nel 1960 e poi ripetere nel 2018, **per vedere se la correlazione tra le due variabili è cambiata**.

In [ ]:
# facciamo il grafico relativo al 1960:
x = np.log(emissions['population'][(emissions['year'] == 1960)])
y = np.log(emissions['emissions_per_capita'][(emissions['year'] == 1960)])
m, b = np.polyfit(x, y, 1)
plt.scatter(x, y, alpha=0.5)
plt.plot(x, m*x + b)
plt.xlabel("Popolazione")
plt.ylabel("Emissioni CO2 pro capite")

In [ ]:
# lo replichiamo per il 2018:
x = np.log(emissions['population'][(emissions['year'] == 2018)])
y = np.log(emissions['emissions_per_capita'][(emissions['year'] == 2018)])
m, b = np.polyfit(x, y, 1)
plt.scatter(x, y, alpha=0.5)
plt.plot(x, m*x + b)
plt.xlabel("Popolazione")
plt.ylabel("Emissioni CO2 pro capite")

**Cosa notate?** 🎤 a voi!